In [1]:
""""Do the necessary imports"""
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os

In [51]:
"""Scrape a first time to all the months where data about the tides can be found and create a list that 
can be pluged in the url to load all the pages where scrapable data is available"""

url = "https://marine.meteoconsult.fr/meteo-marine/horaires-des-marees/pointe-d-agon-944/juillet-2022"

response=requests.get(url)

soup1 = BeautifulSoup(response.content, 'html.parser')

list_dates=[]
list_years=[]

for i in soup1.find_all(class_="month-container"):
    list_dates.append((i.find(class_="name").text))
    list_years.append((i.find(class_="year").text))
    
list_dates_and_years = [m+str(n) for m,n in zip(list_dates, list_years)]

month_years_final=[]
for i in list_dates_and_years:
    i=i.replace("20", "-20")
    month_years_final.append(i)
    





In [77]:
"""Loop on each month and extract the information"""

url = "https://marine.meteoconsult.fr/meteo-marine/horaires-des-marees/pointe-d-agon-944"

days=[]

for month in month_years_final:
      
    req=requests.get(url + f"/{month}")
    soup = BeautifulSoup(req.content, 'html.parser')
    
    for i in soup.find_all(class_="tide-date week"):
        days.append(i.text)
    

    

In [81]:
"""Do the necessary reshapes"""

days_formated=[]
for i in days:
    i=i.replace("\n","")
    days_formated.append(i)

In [82]:
days_formated

['mercredi 6',
 'jeudi 7',
 'vendredi 8',
 'lundi 11',
 'mardi 12',
 'mercredi 13',
 'jeudi 14',
 'vendredi 15',
 'lundi 18',
 'mardi 19',
 'mercredi 20',
 'jeudi 21',
 'vendredi 22',
 'lundi 25',
 'mardi 26',
 'mercredi 27',
 'jeudi 28',
 'vendredi 29',
 'lundi 1',
 'mardi 2',
 'mercredi 3',
 'jeudi 4',
 'vendredi 5',
 'lundi 8',
 'mardi 9',
 'mercredi 10',
 'jeudi 11',
 'vendredi 12',
 'lundi 15',
 'mardi 16',
 'mercredi 17',
 'jeudi 18',
 'vendredi 19',
 'lundi 22',
 'mardi 23',
 'mercredi 24',
 'jeudi 25',
 'vendredi 26',
 'lundi 29',
 'mardi 30',
 'mercredi 31',
 'jeudi 1',
 'vendredi 2',
 'lundi 5',
 'mardi 6',
 'mercredi 7',
 'jeudi 8',
 'vendredi 9',
 'lundi 12',
 'mardi 13',
 'mercredi 14',
 'jeudi 15',
 'vendredi 16',
 'lundi 19',
 'mardi 20',
 'mercredi 21',
 'jeudi 22',
 'vendredi 23',
 'lundi 26',
 'mardi 27',
 'mercredi 28',
 'jeudi 29',
 'vendredi 30',
 'lundi 3',
 'mardi 4',
 'mercredi 5',
 'jeudi 6',
 'vendredi 7',
 'lundi 10',
 'mardi 11',
 'mercredi 12',
 'jeudi 13',